# ML Trader

In [1]:
from utils.packages import *
from utils.variables import *
from utils.i_o import *
from utils.dir_slope import *
from utils.engine import *
from utils.loops import *
from utils.order import *
from utils.ml import *
import winsound

# Notes
---

## Observations

1. No need to make 10 transactions a day. Just one is enough

1. lema or sema match
    1. Using lema match works fine in both opening and closing positions
    1. Using lema has better results
    1. 144 produces better resuls over higher numbers like 1440 

1. Lema angle in order and close
    1. Using lema angle in order, delays order, but overall profits are high
    1. Using lema angle in close also increases overall profit

1. Before everything need to find why 2021 is consistently producing negative results in all the setups.
    1. What is that this causing this issue, how can it be addressed with in the same system
    1. less complex method is essential
    
---


## To Test
1. Find whether the opening or closing that is causing the most issue in loss years
1. Test other years to understand how well this setup works.
1. Remove lema match check from closing. This is delaying bad trasactions
1. Test 5 round up on lema match. Transactions are opening in lema match gaps
1. Ideation
    1. Test lema Bollinger bands to understand flow direction
    1. Test overall flow direction
    
---

## Result by Years

| Year | Angle open | Angle close | Lema match open | Lema match close | Result type | pips  |
| ---- | ---------- | ----------- | --------------- | ---------------- | ----------- | ----- |
| 2022 | Yes        | No          | Yes             | Yes              | Profit      | 200   |
| 2021 | Yes        | No          | Yes             | Yes              | Loss        | \-800 |
| 2020 | Yes        | No          | Yes             | Yes              | Profit      | 800   |
| 2019 | Yes        | No          | Yes             | Yes              | Loss        | \-400 |
| 2018 | Yes        | No          | Yes             | Yes              | Profit      | 300   |
| 2017 | Yes        | No          | Yes             | Yes              | Profit      | 14    |    

data                = get_date_list(data)
data                = read_data(data)

In [3]:
%%time
data = run_engine(data)

pl   -0.07321
dtype: float64
--------------------------------------
            DateTime_frmt  order_side  close_type       pl
70792 2021-12-15 20:25:00         NaN  sema_close -0.00246
71318 2021-12-17 16:15:00         NaN  sema_close -0.00166
71588 2021-12-20 14:45:00         NaN  sema_close -0.00255
71782 2021-12-21 06:55:00         NaN  sema_close -0.00178
72078 2021-12-22 07:35:00         NaN  sema_close -0.00260
72137 2021-12-22 12:30:00         NaN  sema_close -0.00388
72460 2021-12-23 15:25:00         NaN  sema_close  0.00089
72594 2021-12-24 02:35:00         NaN  sema_close -0.00018
72716 2021-12-24 12:45:00         NaN  sema_close -0.00033
73120 2021-12-27 22:30:00         NaN  sema_close -0.00234
73163 2021-12-28 02:05:00         NaN  sema_close -0.00089
73624 2021-12-29 16:30:00         NaN  sema_close -0.00480
73773 2021-12-30 04:55:00         NaN  sema_close -0.00084
73968 2021-12-30 21:10:00         NaN  sema_close -0.00227
74177 2021-12-31 14:35:00         NaN  sema_clo

100%|███████████████████████████████████████| 413/413 [00:00<00:00, 996.11it/s]

Wall time: 1min 56s


In [4]:
x = data['df_ohlc'].groupby(['month_val'])['pl'].sum()
print(x)
print('---------------------')
print(x.sort_values())
print('---------------------')
print(f'max : {x.idxmax()} : {max(x)}')
print(f'min : {x.idxmin()} : {min(x)}')
print('---------------------')
monthly_avg = np.round(data['df_ohlc']['pl'].sum()/len(set(data['df_ohlc']['month_val'])),4)
print(f'monthly_avg : {monthly_avg}')

month_val
1    -0.00457
2     0.01460
3    -0.00086
4    -0.02329
5     0.00664
6    -0.00189
7    -0.01088
8    -0.00155
9    -0.02344
10    0.00566
11    0.00749
12   -0.04112
Name: pl, dtype: float64
---------------------
month_val
12   -0.04112
9    -0.02344
4    -0.02329
7    -0.01088
1    -0.00457
6    -0.00189
8    -0.00155
3    -0.00086
10    0.00566
5     0.00664
11    0.00749
2     0.01460
Name: pl, dtype: float64
---------------------
max : 2 : 0.014599999999999998
min : 12 : -0.04112000000000001
---------------------
monthly_avg : -0.0061


In [5]:
data['lema_gap_pl_analysis_month'] = 8

data = lema_gap_pl_analysis(data)
data['lema_gap_pl_report']

,index,positive,negative,count_diff,positive_sum,negative_sum,sum_diff
3,0.000,1.0,1,0.0,0.00296,-0.00191,0.00105
0,0.001,1.0,5,-4.0,0.00713,-0.00860,-0.00147
1,0.002,1.0,5,-4.0,0.01135,-0.01243,-0.00108
2,0.004,0.0,1,-1.0,0.00000,-0.00158,-0.00158


In [6]:
sys.exit()

SystemExit: 

In [ ]:
print(data['df_small'].groupby(['weekday', 'pl_type'])['pl_type'].count().unstack())
print('----------------------------------------------')
print(data['df_small'].groupby(['weekday', 'pl_type'])['pl'].sum().unstack())

In [ ]:
data['lema_gap_pl_analysis_month'] = 3

data = lema_gap_pl_analysis(data)
data['lema_gap_pl_report']

In [ ]:
y = data['df_ohlc']['pl'].sum()
x = data['df_ohlc'].groupby(['month_val'])['pl'].sum()
send_telegram_message(f'{data["input_year"]}')
send_telegram_message(f'{y}')
send_telegram_message(f'{x}')
send_telegram_message(f'-------------------')

In [ ]:
y = data['df_ohlc'][data['df_ohlc']['month_val'] == 6]
x = y.groupby(['date_val'])['pl'].sum()
print(x)
print('---------------------')
print(f'max : {x.idxmax()} : {max(x)}')
print(f'min : {x.idxmin()} : {min(x)}')
print('---------------------')

In [ ]:
data['df_ohlc'].tail()

In [ ]:
data['df_ohlc']['lema_gap'].describe()

In [ ]:
x = 5
data['df_ohlc'].iloc[-5:,x:x+15]

In [ ]:
data['df_small']['lema_gap'].describe()

In [ ]:
a = np.arange(0,20)
for b in a:
    x = int(round(b/5.0)*5.0)
    print(b, x)

In [ ]:
abs(all['negative_sum']) / all['positive_sum']

In [ ]:
df_small['order_side']

In [ ]:
data['df_small'].to_csv('data/temp.csv')

In [8]:
data["plot"] = True
data['plot_type'] = 'file'
data['reduce_plot'] = True
month = 12

# Monthly Chart -------------------
data['plot_start']  = f"{data['input_year']}-{month}-1"

if month == 12:
    data['plot_stop']   = f"{data['input_year']+1}-1-1"
else:
    data['plot_stop']   = f"{data['input_year']}-{month+1}-1"

# Year Chart ----------------------------
# data['plot_start']  = f"{data['input_year']}-1-1"
# data['plot_stop']   = f"{data['input_year']+1}-1-1"
# Year Chart ----------------------------

# data['things_to_plot'] = ['tick', 'ema', 'BBands', 'dots']
# data['things_to_plot'] = ['ema', 'BBands', 'dots', 'indicators']
# data['things_to_plot'] = ['ema', 'BBands', 'dots', 'cross']
# data['things_to_plot'] = ['ema', 'BBands', 'dots', 'positions', 'tip']
# data['things_to_plot'] = ['ema', 'BBands', 'dots', 'positions', 'lema_angle']
# data['things_to_plot'] = ['ema', 'sema_angle', 'slema_angle', 'lema_angle', 'dir']
# data['things_to_plot'] = ['ema', 'positions', 'lema_angle','slema_angle','sema_angle', 'rsi']
# data['things_to_plot'] = ['ema','positions', 'lema_angle','lema_diff', 'lema_max_min']
# data['things_to_plot'] = ['candles','sema', 'slema','positions','lema', 'lema_angle', 'lema_max_min', 'close_diff']
# data['things_to_plot'] = ['lema', 'tip', 'candles', 'lema_angle', 'lema_change']
data['things_to_plot'] = ['candles','sema', 'slema','lema', 'lema_match', 'positions', 'lema_change']

plot_graph(data)

generate_result_report(data)